---
## 🔧 Krok 0: Import biblioteki DLT

---
## 🌟 Star Schema Model - What We're Building and Why

### Source Data (OLTP - Transactional Systems)

```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│   Customers     │     │ SalesOrderHeader │     │    Products     │
├─────────────────┤     ├──────────────────┤     ├─────────────────┤
│ CustomerID      │◄────│ CustomerID       │     │ ProductID       │
│ FirstName       │     │ SalesOrderID     │────►│ Name            │
│ LastName        │     │ OrderDate        │     │ ListPrice       │
│ EmailAddress    │     │ TotalDue         │     │ ProductCategory │
│ CompanyName     │     │ Status           │     └─────────────────┘
│ ModifiedDate    │     └────────┬─────────┘
└─────────────────┘              │
                                 │
┌─────────────────┐              │
│SalesOrderDetail │◄─────────────┘
├─────────────────┤
│ SalesOrderID    │
│ ProductID       │
│ OrderQty        │
│ UnitPrice       │
│ LineTotal       │
└─────────────────┘
```

### ❌ Problems with OLTP Data:
- **Multiple JOINs** needed for simple reports
- **Normalization** = data scattered across many tables
- **Slow analytical queries**
- **No change history** - we don't know what the customer looked like a year ago

---

### ✅ Solution: Star Schema

```
                    ┌─────────────────────┐
                    │    Dim_Date         │
                    │ (SCD Type 1)        │
                    ├─────────────────────┤
                    │ date_key (SK)       │
                    │ full_date           │
                    │ year, month, day    │
                    │ quarter             │
                    │ day_of_week         │
                    └──────────┬──────────┘
                               │
┌─────────────────────┐        │        ┌─────────────────────┐
│   Dim_Customer      │        │        │    Dim_Product      │
│   (SCD Type 2)      │        │        │    (SCD Type 1)     │
├─────────────────────┤        │        ├─────────────────────┤
│ customer_key (SK)   │        │        │ product_key (SK)    │
│ customer_id (NK)    │        │        │ product_id (NK)     │
│ full_name           │        │        │ product_name        │
│ email               │        │        │ category_name       │
│ company_name        │◄───────┼───────►│ list_price          │
│ is_current          │        │        │ color               │
│ valid_from          │        │        └─────────────────────┘
│ valid_to            │        │
└─────────────────────┘        │
                               │
              ┌────────────────┴────────────────┐
              │         FACT_SALES              │
              │       (Fact Table)              │
              ├─────────────────────────────────┤
              │ sales_key (SK)                  │
              │ customer_key (FK) ──────────────┼──► Dim_Customer
              │ product_key (FK) ───────────────┼──► Dim_Product
              │ date_key (FK) ──────────────────┼──► Dim_Date
              │                                 │
              │ order_id (DD)                   │
              │ quantity                        │
              │ unit_price                      │
              │ line_total                      │
              │ total_due                       │
              └─────────────────────────────────┘

Legend:
  SK = Surrogate Key (artificial key)
  NK = Natural Key (business key)
  FK = Foreign Key
  DD = Degenerate Dimension
```

---

### 🎯 Star Schema Benefits:

| Aspect | OLTP (before) | Star Schema (after) |
|--------|--------------|---------------------|
| **Queries** | 5-6 JOINs | 1-2 JOINs |
| **Performance** | Slow | Very fast |
| **Understandability** | Complex | Intuitive |
| **Change history** | None | SCD Type 2 |
| **Aggregations** | Expensive | Pre-computed |

---
## 🔄 Slowly Changing Dimensions (SCD) - Theory

### What is SCD?

**SCD** (Slowly Changing Dimensions) are techniques for managing changes in data warehouse dimensions.

Imagine customer **John Smith** changed his last name to **John Johnson**. What do we do?

---

### 📝 SCD Type 1 - Overwrite

**Rule:** Simply overwrite the old value with the new one.

```
BEFORE:                         AFTER:
┌────────────┬─────────────┐    ┌────────────┬─────────────┐
│ CustomerID │ LastName    │    │ CustomerID │ LastName    │
├────────────┼─────────────┤    ├────────────┼─────────────┤
│ 1          │ Smith       │ => │ 1          │ Johnson     │
└────────────┴─────────────┘    └────────────┴─────────────┘
```

**When to use:**
- ✅ Data where history **IS NOT important** (e.g., typo corrections)
- ✅ **Static** dimensions (e.g., product categories)
- ✅ When you want to **save space**

**Examples in Adventure Works:**
- `Dim_Product` - product price, name
- `Dim_Date` - calendar doesn't change

---

### 📜 SCD Type 2 - History Tracking

**Rule:** Add a new row, keep the old one with validity dates.

```
BEFORE:
┌─────────────┬────────────┬─────────────┬────────────┬────────────┬────────────┐
│ customer_key│ CustomerID │ LastName    │ is_current │ valid_from │ valid_to   │
├─────────────┼────────────┼─────────────┼────────────┼────────────┼────────────┤
│ 1           │ 1          │ Smith       │ true       │ 2023-01-01 │ 9999-12-31 │
└─────────────┴────────────┴─────────────┴────────────┴────────────┴────────────┘

AFTER CHANGE (2024-06-15):
┌─────────────┬────────────┬─────────────┬────────────┬────────────┬────────────┐
│ customer_key│ CustomerID │ LastName    │ is_current │ valid_from │ valid_to   │
├─────────────┼────────────┼─────────────┼────────────┼────────────┼────────────┤
│ 1           │ 1          │ Smith       │ false      │ 2023-01-01 │ 2024-06-14 │
│ 2           │ 1          │ Johnson     │ true       │ 2024-06-15 │ 9999-12-31 │
└─────────────┴────────────┴─────────────┴────────────┴────────────┴────────────┘
```

**When to use:**
- ✅ Data where history **IS important** for analysis
- ✅ **Dynamic** dimensions (e.g., customer data, addresses)
- ✅ When you need **historical analysis** ("what was the customer like a year ago?")

**Examples in Adventure Works:**
- `Dim_Customer` - company name, email may change

---

### ⚖️ SCD Type 1 vs Type 2 Comparison

| Aspect | SCD Type 1 | SCD Type 2 |
|--------|------------|------------|
| **History** | ❌ Lost | ✅ Preserved |
| **Complexity** | Simple | More complex |
| **Data size** | Small | Grows over time |
| **Queries** | Simple | Require `is_current = true` |
| **DW usage** | Static dimensions | Dynamic dimensions |

---

### 🔧 Implementation with Delta Lake MERGE

Delta Lake offers powerful `MERGE` command for SCD implementation:

```sql
-- SCD Type 1 (overwrite)
MERGE INTO dim_product AS target
USING source_product AS source
ON target.product_id = source.product_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

-- SCD Type 2 (history) - requires 2 steps
-- 1. Close old records
-- 2. Insert new records
```

---
---

# 🛠️ HANDS-ON SECTION

## ⚙️ Step 0: Environment Setup

**NOTE:** This workshop is **self-contained** - it doesn't require completing previous workshops.

In [ ]:
# Run the setup notebook first to set variables!
# %run ../00_setup

# Configuration (will be overwritten by 00_setup)
catalog = spark.conf.get("spark.databricks.unityCatalog.catalog", "training")
schema = spark.conf.get("spark.databricks.unityCatalog.schema", "workshop")
volume_path = f"/Volumes/{catalog}/{schema}/data"

# Table prefix for this workshop (isolation from others)
TABLE_PREFIX = "w3_"

print("=" * 60)
print("🏭 WORKSHOP 3: LAKEFLOW PIPELINE - STAR SCHEMA")
print("=" * 60)
print(f"📂 Catalog: {catalog}")
print(f"📂 Schema: {schema}")
print(f"📂 Volume Path: {volume_path}")
print(f"📂 Table Prefix: {TABLE_PREFIX}")
print("=" * 60)

---
## 🥉 Step 1: Bronze Layer - Load Raw Data

The first step in the Medallion Architecture is loading **raw data** into the Bronze layer.
Data is saved **without transformations** - exactly as it came from the source.

### Source Files:
| File | Description | Records |
|------|-------------|---------|
| `Customers.csv` | Customer data | ~10,000 |
| `Product.csv` | Products | ~500 |
| `ProductCategory.csv` | Product categories | ~42 |
| `SalesOrderHeader.csv` | Order headers | ~50,000 |
| `SalesOrderDetail.csv` | Order line items | ~129,000 |

In [ ]:
# ============================================================
# 🥉 BRONZE LAYER - Load Raw Data
# ============================================================

# List of source files
source_files = [
    "Customers.csv",
    "Product.csv",
    "ProductCategory.csv",
    "SalesOrderHeader.csv",
    "SalesOrderDetail.csv"
]

# Load all CSV files into Bronze Delta tables
for file_name in source_files:
    table_name = f"{TABLE_PREFIX}bronze_{file_name.replace('.csv', '').lower()}"
    file_path = f"{volume_path}/{file_name}"
    
    print(f"📥 Loading: {file_name} -> {table_name}")
    
    # Read CSV
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(file_path)
    
    # Save as Delta table (Bronze)
    df.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable(f"{catalog}.{schema}.{table_name}")
    
    print(f"   ✅ Saved {df.count()} records")

print("\n" + "=" * 60)
print("✅ BRONZE LAYER COMPLETE!")
print("=" * 60)

In [ ]:
# Verify Bronze Layer
print("📊 BRONZE LAYER SUMMARY:")
print("-" * 50)

for file_name in source_files:
    table_name = f"{TABLE_PREFIX}bronze_{file_name.replace('.csv', '').lower()}"
    count = spark.table(f"{catalog}.{schema}.{table_name}").count()
    print(f"  {table_name}: {count:,} records")

# Preview sample data
print("\n📋 Sample Data - Bronze Customers:")
display(spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_customers").limit(3))

---
## 🥈 Step 2: Silver Layer - Clean Data

The Silver layer contains **cleaned and normalized** data:
- ✅ Correct data types
- ✅ Removed duplicates
- ✅ Filled missing values
- ✅ Data quality validation

### Task 2.1: Create Silver Customers

**Requirements:**
1. Select only needed columns
2. Create `full_name` column (FirstName + LastName)
3. Remove duplicates by `CustomerID`
4. Filter records without `EmailAddress`

In [ ]:
# ============================================================
# 🥈 SILVER LAYER - Cleaned Data
# ============================================================

from pyspark.sql.functions import *
from pyspark.sql.types import *

# --- SILVER CUSTOMERS ---
print("🔧 Creating: Silver Customers")

silver_customers = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_customers") \
    .select(
        col("CustomerID").cast("int"),
        "FirstName",
        "LastName", 
        "EmailAddress",
        "Phone",
        "CompanyName",
        col("ModifiedDate").cast("timestamp")
    ) \
    .withColumn("full_name", concat_ws(" ", col("FirstName"), col("LastName"))) \
    .filter(col("EmailAddress").isNotNull()) \
    .dropDuplicates(["CustomerID"])

silver_customers.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}silver_customers")

print(f"   ✅ Silver Customers: {silver_customers.count()} records")

In [ ]:
# --- SILVER PRODUCTS (with category) ---
print("🔧 Creating: Silver Products")

bronze_products = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_product")
bronze_categories = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_productcategory")

silver_products = bronze_products \
    .join(bronze_categories, bronze_products.ProductCategoryID == bronze_categories.ProductCategoryID, "left") \
    .select(
        bronze_products.ProductID.cast("int"),
        bronze_products.Name.alias("product_name"),
        bronze_products.ProductNumber,
        bronze_products.Color,
        bronze_products.ListPrice.cast("decimal(18,2)"),
        bronze_products.StandardCost.cast("decimal(18,2)"),
        bronze_categories.Name.alias("category_name"),
        bronze_products.ModifiedDate.cast("timestamp")
    ) \
    .filter(col("ProductID").isNotNull()) \
    .dropDuplicates(["ProductID"])

silver_products.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}silver_products")

print(f"   ✅ Silver Products: {silver_products.count()} records")

In [ ]:
# --- SILVER ORDERS (Header + Detail) ---
print("🔧 Creating: Silver Orders")

bronze_orders = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_salesorderheader")
bronze_details = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_salesorderdetail")

# Silver Order Header
silver_order_header = bronze_orders \
    .select(
        col("SalesOrderID").cast("int"),
        col("CustomerID").cast("int"),
        col("ShipToAddressID").cast("int"),
        col("OrderDate").cast("date"),
        col("ShipDate").cast("date"),
        col("TotalDue").cast("decimal(18,2)"),
        col("Status").cast("int"),
        "ShipMethod"
    ) \
    .filter(col("SalesOrderID").isNotNull())

silver_order_header.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}silver_order_header")

# Silver Order Detail
silver_order_detail = bronze_details \
    .select(
        col("SalesOrderID").cast("int"),
        col("SalesOrderDetailID").cast("int"),
        col("ProductID").cast("int"),
        col("OrderQty").cast("int"),
        col("UnitPrice").cast("decimal(18,2)"),
        col("UnitPriceDiscount").cast("decimal(5,2)"),
        col("LineTotal").cast("decimal(18,2)")
    ) \
    .filter(col("SalesOrderDetailID").isNotNull())

silver_order_detail.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}silver_order_detail")

print(f"   ✅ Silver Order Header: {silver_order_header.count()} records")
print(f"   ✅ Silver Order Detail: {silver_order_detail.count()} records")

print("\n" + "=" * 60)
print("✅ SILVER LAYER COMPLETE!")
print("=" * 60)

---
---

## 🥇 Step 3: Gold Layer - Dimensions with SCD Type 1

### Dim_Product (SCD Type 1)

The product dimension uses **SCD Type 1** - when the price or name changes, we simply **overwrite** the old value.

**Why SCD Type 1?**
- Products rarely change
- We don't need product price history in the dimension (can track separately)
- Simpler model = faster queries

### Task 3.1: Create Dim_Product

**PySpark Version:**

In [ ]:
# ============================================================
# 🥇 GOLD LAYER - Dim_Product (SCD Type 1) - PySpark
# ============================================================

print("🌟 Creating: Dim_Product (SCD Type 1)")

# Load source data from Silver
silver_products = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}silver_products")

# Create dimension with Surrogate Key
dim_product = silver_products \
    .withColumn("product_key", monotonically_increasing_id() + 1) \
    .select(
        col("product_key"),
        col("ProductID").alias("product_id"),  # Natural Key
        col("product_name"),
        col("ProductNumber").alias("product_number"),
        col("Color").alias("color"),
        col("category_name"),
        col("ListPrice").alias("list_price"),
        col("StandardCost").alias("standard_cost"),
        current_timestamp().alias("load_timestamp")
    )

# Save as Delta table
dim_product.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}dim_product")

print(f"   ✅ Dim_Product: {dim_product.count()} records")
display(dim_product.limit(5))

**SQL Version (equivalent):**

In [ ]:
# ============================================================
# 🥇 GOLD LAYER - Dim_Product (SCD Type 1) - SQL Version
# ============================================================

# Same logic in SQL
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}dim_product_sql AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY ProductID) AS product_key,
    ProductID AS product_id,
    product_name,
    ProductNumber AS product_number,
    Color AS color,
    category_name,
    ListPrice AS list_price,
    StandardCost AS standard_cost,
    current_timestamp() AS load_timestamp
FROM {catalog}.{schema}.{TABLE_PREFIX}silver_products
""")

print("✅ Dim_Product (SQL) created!")

# SCD Type 1 Update Pattern - when new data arrives
print("""
📝 SCD Type 1 - Update Pattern (MERGE):

MERGE INTO dim_product AS target
USING silver_products AS source
ON target.product_id = source.ProductID
WHEN MATCHED AND (
    target.product_name != source.product_name OR
    target.list_price != source.ListPrice
) THEN UPDATE SET
    target.product_name = source.product_name,
    target.list_price = source.ListPrice,
    target.load_timestamp = current_timestamp()
WHEN NOT MATCHED THEN INSERT (
    product_key, product_id, product_name, product_number, 
    color, category_name, list_price, standard_cost, load_timestamp
) VALUES (
    (SELECT COALESCE(MAX(product_key), 0) + 1 FROM dim_product),
    source.ProductID, source.product_name, source.ProductNumber,
    source.Color, source.category_name, source.ListPrice, 
    source.StandardCost, current_timestamp()
)
""")

---
### Dim_Date (SCD Type 1)

The date dimension is a **static dimension** - the calendar doesn't change.
We generate it once and use it repeatedly.

### Task 3.2: Create Dim_Date

In [ ]:
# ============================================================
# 🥇 GOLD LAYER - Dim_Date (SCD Type 1) - PySpark
# ============================================================

from datetime import datetime, timedelta

print("📅 Creating: Dim_Date")

# Generate date range (2022-2026)
start_date = datetime(2022, 1, 1)
end_date = datetime(2026, 12, 31)
date_range = [(start_date + timedelta(days=x)).strftime('%Y-%m-%d') 
              for x in range((end_date - start_date).days + 1)]

# Create DataFrame with dates
dim_date = spark.createDataFrame([(d,) for d in date_range], ["full_date"]) \
    .withColumn("full_date", col("full_date").cast("date")) \
    .withColumn("date_key", date_format(col("full_date"), "yyyyMMdd").cast("int")) \
    .withColumn("year", year(col("full_date"))) \
    .withColumn("month", month(col("full_date"))) \
    .withColumn("day", dayofmonth(col("full_date"))) \
    .withColumn("quarter", quarter(col("full_date"))) \
    .withColumn("day_of_week", dayofweek(col("full_date"))) \
    .withColumn("day_name", date_format(col("full_date"), "EEEE")) \
    .withColumn("month_name", date_format(col("full_date"), "MMMM")) \
    .withColumn("is_weekend", when(dayofweek(col("full_date")).isin([1, 7]), True).otherwise(False)) \
    .select(
        "date_key",
        "full_date",
        "year",
        "month",
        "month_name",
        "day",
        "quarter",
        "day_of_week",
        "day_name",
        "is_weekend"
    )

# Save as Delta table
dim_date.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}dim_date")

print(f"   ✅ Dim_Date: {dim_date.count()} records (5 years)")
display(dim_date.limit(5))

---
---

## 🥇 Step 4: Gold Layer - Dimensions with SCD Type 2

### Dim_Customer (SCD Type 2)

The customer dimension uses **SCD Type 2** - when a customer changes data (e.g., company, email), we **preserve history**.

**Why SCD Type 2?**
- We want to know which company the customer belonged to **at the time of the order**
- Historical analysis requires past state of data
- Audit and compliance require tracking changes

### SCD Type 2 Structure:
| Column | Description |
|--------|-------------|
| `customer_key` | Surrogate Key (unique for each version) |
| `customer_id` | Natural Key (business customer ID) |
| `is_current` | Is this the current version? |
| `valid_from` | Validity start date |
| `valid_to` | Validity end date (9999-12-31 = current) |

### Task 4.1: Create Dim_Customer with SCD Type 2

**PySpark Version:**

In [ ]:
# ============================================================
# 🥇 GOLD LAYER - Dim_Customer (SCD Type 2) - PySpark
# ============================================================

print("👤 Creating: Dim_Customer (SCD Type 2)")

# Load source data
silver_customers = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}silver_customers")

# Create dimension with SCD Type 2 fields
dim_customer = silver_customers \
    .withColumn("customer_key", monotonically_increasing_id() + 1) \
    .withColumn("is_current", lit(True)) \
    .withColumn("valid_from", current_date()) \
    .withColumn("valid_to", lit("9999-12-31").cast("date")) \
    .select(
        col("customer_key"),
        col("CustomerID").alias("customer_id"),  # Natural Key
        col("full_name"),
        col("FirstName").alias("first_name"),
        col("LastName").alias("last_name"),
        col("EmailAddress").alias("email"),
        col("Phone").alias("phone"),
        col("CompanyName").alias("company_name"),
        col("is_current"),
        col("valid_from"),
        col("valid_to")
    )

# Save as Delta table
dim_customer.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}dim_customer")

print(f"   ✅ Dim_Customer: {dim_customer.count()} records")
display(dim_customer.limit(5))

### 🔄 SCD Type 2 - Update with MERGE (Simulation)

Now let's simulate a customer data update and show how SCD Type 2 works:

In [ ]:
# ============================================================
# 🔄 SCD TYPE 2 - Update with Delta MERGE
# ============================================================

# Simulate customer data change - customer changed company
print("📝 Simulating customer data change...")

# Get a sample customer
sample_customer = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}dim_customer") \
    .filter(col("is_current") == True) \
    .limit(1).collect()[0]

print(f"   Before change: {sample_customer['full_name']} - {sample_customer['company_name']}")

# Create "new data" - simulate company change
updated_data = [(
    sample_customer['customer_id'],
    sample_customer['full_name'],
    sample_customer['first_name'],
    sample_customer['last_name'],
    sample_customer['email'],
    sample_customer['phone'],
    "NEW COMPANY INC."  # Changed company!
)]

updated_df = spark.createDataFrame(updated_data, [
    "customer_id", "full_name", "first_name", "last_name", 
    "email", "phone", "company_name"
])

print(f"   New company: NEW COMPANY INC.")

In [ ]:
# ============================================================
# SCD TYPE 2 - MERGE in SQL
# ============================================================

# Register DataFrame as temp view
updated_df.createOrReplaceTempView("updated_customers")

# SCD Type 2 requires 2 steps:
# Step 1: Close old records (set valid_to and is_current = false)
# Step 2: Insert new records

dim_customer_table = f"{catalog}.{schema}.{TABLE_PREFIX}dim_customer"

# STEP 1: Close old records
spark.sql(f"""
MERGE INTO {dim_customer_table} AS target
USING updated_customers AS source
ON target.customer_id = source.customer_id 
   AND target.is_current = true
   AND target.company_name != source.company_name
WHEN MATCHED THEN UPDATE SET
    target.is_current = false,
    target.valid_to = current_date() - INTERVAL 1 DAY
""")

print("✅ Step 1: Closed old records")

# STEP 2: Insert new records
spark.sql(f"""
INSERT INTO {dim_customer_table}
SELECT 
    (SELECT COALESCE(MAX(customer_key), 0) + 1 FROM {dim_customer_table}) AS customer_key,
    source.customer_id,
    source.full_name,
    source.first_name,
    source.last_name,
    source.email,
    source.phone,
    source.company_name,
    true AS is_current,
    current_date() AS valid_from,
    DATE '9999-12-31' AS valid_to
FROM updated_customers source
WHERE EXISTS (
    SELECT 1 FROM {dim_customer_table} target 
    WHERE target.customer_id = source.customer_id 
    AND target.is_current = false
    AND target.valid_to = current_date() - INTERVAL 1 DAY
)
""")

print("✅ Step 2: Inserted new records")

# Verification
print("\n📊 Change history for customer:")
display(spark.sql(f"""
SELECT customer_key, customer_id, full_name, company_name, 
       is_current, valid_from, valid_to
FROM {dim_customer_table}
WHERE customer_id = {sample_customer['customer_id']}
ORDER BY valid_from
"""))

---
## 7. Gold Layer - Fact Table

### Fact_Sales - Central Fact Table

Now we build the central fact table that connects all dimensions. This table contains:
- **Measures**: quantity, unit_price, line_total, order_total
- **Foreign Keys**: customer_key, product_key, date_key

The fact table follows a **star schema pattern** where each foreign key points to a dimension table.

In [ ]:
# ============================================================
# GOLD LAYER: Fact_Sales
# ============================================================

from pyspark.sql.functions import (
    col, date_format, sha2, concat_ws, monotonically_increasing_id
)

# Read silver tables
silver_orders = spark.table("silver_orders")
silver_order_details = spark.table("silver_order_details")

# Read dimension tables for key lookup
dim_customer = spark.table("gold_dim_customer").filter(col("is_current") == True)
dim_product = spark.table("gold_dim_product")
dim_date = spark.table("gold_dim_date")

# Build fact table by joining with dimensions
fact_sales = (
    silver_order_details
    .join(silver_orders, "SalesOrderID")
    # Join to get dimension keys
    .join(
        dim_customer.select("customer_key", "customer_id"),
        silver_orders["CustomerID"] == dim_customer["customer_id"],
        "left"
    )
    .join(
        dim_product.select("product_key", "product_id"),
        silver_order_details["ProductID"] == dim_product["product_id"],
        "left"
    )
    .join(
        dim_date.select("date_key", "full_date"),
        date_format(silver_orders["OrderDate"], "yyyy-MM-dd") == date_format(dim_date["full_date"], "yyyy-MM-dd"),
        "left"
    )
    # Select final columns
    .select(
        monotonically_increasing_id().alias("sales_fact_id"),
        col("SalesOrderID").alias("order_id"),
        col("SalesOrderDetailID").alias("order_line_id"),
        col("customer_key"),
        col("product_key"),
        col("date_key"),
        col("OrderQty").alias("quantity"),
        col("UnitPrice").alias("unit_price"),
        col("LineTotal").alias("line_total"),
        col("TotalDue").alias("order_total"),
        col("OrderDate").alias("order_date"),
        col("ShipDate").alias("ship_date")
    )
)

# Save as Delta table
fact_sales.write.format("delta").mode("overwrite").saveAsTable("gold_fact_sales")

print(f"✅ Fact_Sales created with {fact_sales.count():,} records")
display(fact_sales.limit(10))

In [ ]:
-- SQL Equivalent: Create Fact_Sales
-- This shows the same logic using pure SQL

CREATE OR REPLACE TABLE gold_fact_sales_sql AS
SELECT 
    monotonically_increasing_id() as sales_fact_id,
    sod.SalesOrderID as order_id,
    sod.SalesOrderDetailID as order_line_id,
    dc.customer_key,
    dp.product_key,
    dd.date_key,
    sod.OrderQty as quantity,
    sod.UnitPrice as unit_price,
    sod.LineTotal as line_total,
    so.TotalDue as order_total,
    so.OrderDate as order_date,
    so.ShipDate as ship_date
FROM silver_order_details sod
JOIN silver_orders so ON sod.SalesOrderID = so.SalesOrderID
LEFT JOIN gold_dim_customer dc 
    ON so.CustomerID = dc.customer_id AND dc.is_current = true
LEFT JOIN gold_dim_product dp 
    ON sod.ProductID = dp.product_id
LEFT JOIN gold_dim_date dd 
    ON date_format(so.OrderDate, 'yyyy-MM-dd') = date_format(dd.full_date, 'yyyy-MM-dd');

---
## 8. SCD Type 2 - Simulating Changes

### Understanding the MERGE Pattern for SCD Type 2

When new data arrives, we need to:
1. **Identify changed records** - Compare incoming data with existing current records
2. **Expire old versions** - Set `is_current = false` and `valid_to = current_timestamp`
3. **Insert new versions** - Add new record with `is_current = true` and `version + 1`

Below we simulate customer updates and show the MERGE pattern:

In [ ]:
# ============================================================
# SIMULATING CUSTOMER UPDATES FOR SCD TYPE 2
# ============================================================

from pyspark.sql.functions import col, lit, when, current_timestamp
from delta.tables import DeltaTable

# Simulate incoming customer updates (e.g., customer moved to new city)
# In real scenarios, this would come from a source system
updated_customers_data = [
    (1, "John", "Smith", "john.smith@newmail.com", "New York"),  # Changed email and city
    (2, "Jane", "Doe", "jane.doe@email.com", "Los Angeles"),     # Changed city
    (99999, "New", "Customer", "new@email.com", "Chicago")       # New customer
]

updated_customers_df = spark.createDataFrame(
    updated_customers_data, 
    ["CustomerID", "FirstName", "LastName", "EmailAddress", "City"]
)

print("📥 Incoming customer updates:")
display(updated_customers_df)

In [ ]:
# ============================================================
# SCD TYPE 2 MERGE PATTERN (PySpark with Delta)
# ============================================================

from pyspark.sql.functions import (
    col, lit, current_timestamp, sha2, concat_ws,
    max as spark_max, coalesce
)
from delta.tables import DeltaTable

# Get reference to existing dimension table
dim_customer_table = DeltaTable.forName(spark, "gold_dim_customer")

# Prepare incoming updates with the same structure
incoming_customers = (updated_customers_df
    .withColumn("customer_key", sha2(col("CustomerID").cast("string"), 256))
    .withColumn("full_name", concat_ws(" ", col("FirstName"), col("LastName")))
    .withColumn("is_current", lit(True))
    .withColumn("valid_from", current_timestamp())
    .withColumn("valid_to", lit(None).cast("timestamp"))
    .select(
        col("customer_key"),
        col("CustomerID").alias("customer_id"),
        col("FirstName").alias("first_name"),
        col("LastName").alias("last_name"),
        col("EmailAddress").alias("email"),
        col("City").alias("city"),
        col("full_name"),
        col("is_current"),
        col("valid_from"),
        col("valid_to")
    )
)

# Get max version for each customer to determine next version
current_versions = (
    spark.table("gold_dim_customer")
    .groupBy("customer_id")
    .agg(spark_max("version").alias("max_version"))
)

# Add version to incoming records
incoming_with_version = (
    incoming_customers
    .join(current_versions, "customer_id", "left")
    .withColumn("version", coalesce(col("max_version") + 1, lit(1)))
    .drop("max_version")
)

print("📋 Prepared incoming records with version:")
display(incoming_with_version)

In [ ]:
# ============================================================
# EXECUTE SCD TYPE 2 MERGE
# ============================================================

# The MERGE statement for SCD Type 2:
# 1. When matched AND values changed: expire old record, insert new version
# 2. When not matched: insert as new record

# First, expire the old current records for customers that have updates
dim_customer_table.alias("target").merge(
    incoming_with_version.alias("source"),
    "target.customer_id = source.customer_id AND target.is_current = true"
).whenMatchedUpdate(
    condition = """
        target.email != source.email OR 
        target.city != source.city OR 
        target.first_name != source.first_name OR
        target.last_name != source.last_name
    """,
    set = {
        "is_current": lit(False),
        "valid_to": current_timestamp()
    }
).execute()

print("✅ Step 1: Expired old current records")

# Now insert the new versions
incoming_with_version.write.format("delta").mode("append").saveAsTable("gold_dim_customer")

print("✅ Step 2: Inserted new versions")

# Verify the changes
print("\n📊 Customer history for CustomerID = 1:")
display(
    spark.table("gold_dim_customer")
    .filter(col("customer_id") == 1)
    .orderBy("version")
)

In [ ]:
-- SQL Equivalent: SCD Type 2 MERGE Pattern
-- This shows the same logic using pure SQL

-- Step 1: Expire old records where values have changed
MERGE INTO gold_dim_customer AS target
USING (
    SELECT 
        sha2(CAST(CustomerID AS STRING), 256) as customer_key,
        CustomerID as customer_id,
        FirstName as first_name,
        LastName as last_name,
        EmailAddress as email,
        City as city
    FROM updated_customers_temp
) AS source
ON target.customer_id = source.customer_id AND target.is_current = true
WHEN MATCHED AND (
    target.email != source.email OR 
    target.city != source.city OR
    target.first_name != source.first_name OR
    target.last_name != source.last_name
) THEN
    UPDATE SET 
        is_current = false,
        valid_to = current_timestamp();

-- Step 2: Insert new versions
INSERT INTO gold_dim_customer
SELECT 
    sha2(CAST(CustomerID AS STRING), 256) as customer_key,
    CustomerID as customer_id,
    FirstName as first_name,
    LastName as last_name,
    EmailAddress as email,
    City as city,
    concat(FirstName, ' ', LastName) as full_name,
    true as is_current,
    current_timestamp() as valid_from,
    NULL as valid_to,
    COALESCE(
        (SELECT MAX(version) + 1 FROM gold_dim_customer WHERE customer_id = u.CustomerID),
        1
    ) as version
FROM updated_customers_temp u;

---
## 9. Validation & Analytics Queries

### Validating the Star Schema

Let's run some queries to validate our Star Schema is working correctly:

In [ ]:
-- Validation Query 1: Row counts across all tables
SELECT 'bronze_customers' as table_name, COUNT(*) as row_count FROM bronze_customers
UNION ALL
SELECT 'bronze_products', COUNT(*) FROM bronze_products
UNION ALL
SELECT 'bronze_orders', COUNT(*) FROM bronze_orders
UNION ALL
SELECT 'silver_customers', COUNT(*) FROM silver_customers
UNION ALL
SELECT 'silver_products', COUNT(*) FROM silver_products
UNION ALL
SELECT 'silver_orders', COUNT(*) FROM silver_orders
UNION ALL
SELECT 'gold_dim_customer', COUNT(*) FROM gold_dim_customer
UNION ALL
SELECT 'gold_dim_product', COUNT(*) FROM gold_dim_product
UNION ALL
SELECT 'gold_dim_date', COUNT(*) FROM gold_dim_date
UNION ALL
SELECT 'gold_fact_sales', COUNT(*) FROM gold_fact_sales
ORDER BY table_name;

In [ ]:
-- Validation Query 2: Sample Star Schema query - Sales by Product Category
SELECT 
    dp.category_name,
    dd.year,
    dd.month_name,
    COUNT(DISTINCT f.order_id) as total_orders,
    SUM(f.quantity) as total_quantity,
    ROUND(SUM(f.line_total), 2) as total_revenue
FROM gold_fact_sales f
JOIN gold_dim_product dp ON f.product_key = dp.product_key
JOIN gold_dim_date dd ON f.date_key = dd.date_key
GROUP BY dp.category_name, dd.year, dd.month_name
ORDER BY total_revenue DESC
LIMIT 10;

In [ ]:
-- Validation Query 3: Top customers by revenue
SELECT 
    dc.full_name,
    dc.email,
    dc.city,
    COUNT(DISTINCT f.order_id) as total_orders,
    ROUND(SUM(f.line_total), 2) as total_spent
FROM gold_fact_sales f
JOIN gold_dim_customer dc ON f.customer_key = dc.customer_key
WHERE dc.is_current = true
GROUP BY dc.full_name, dc.email, dc.city
ORDER BY total_spent DESC
LIMIT 10;

In [ ]:
-- Validation Query 4: SCD Type 2 History Check
-- Shows customers with multiple versions (history)
SELECT 
    customer_id,
    full_name,
    email,
    city,
    version,
    is_current,
    valid_from,
    valid_to
FROM gold_dim_customer
WHERE customer_id IN (
    SELECT customer_id 
    FROM gold_dim_customer 
    GROUP BY customer_id 
    HAVING COUNT(*) > 1
)
ORDER BY customer_id, version;

---
## 10. Complete Solutions

This section provides complete end-to-end implementations of the Star Schema pipeline using both **PySpark** and **SQL** approaches.

### Solution A: Complete PySpark Implementation

In [ ]:
# ============================================================
# SOLUTION A: COMPLETE PYSPARK STAR SCHEMA PIPELINE
# ============================================================
# This is the full end-to-end implementation using PySpark DataFrame API

from pyspark.sql.functions import (
    col, lit, current_timestamp, date_format, year, month, dayofmonth,
    dayofweek, quarter, weekofyear, sha2, concat_ws, monotonically_increasing_id,
    when, coalesce, max as spark_max, trim, upper, lower, regexp_replace
)
from delta.tables import DeltaTable

# Configuration
WORKSHOP_PATH = "/Volumes/main/default/workshop_data"
CATALOG = "workshop_catalog"
SCHEMA = "star_schema"

# ============================================================
# STEP 1: BRONZE LAYER - Raw Data Ingestion
# ============================================================

def load_bronze_tables():
    """Load all raw CSV files into Bronze Delta tables"""
    
    bronze_tables = {
        "bronze_customers": f"{WORKSHOP_PATH}/Customers.csv",
        "bronze_products": f"{WORKSHOP_PATH}/Product.csv",
        "bronze_product_category": f"{WORKSHOP_PATH}/ProductCategory.csv",
        "bronze_orders": f"{WORKSHOP_PATH}/SalesOrderHeader.csv",
        "bronze_order_details": f"{WORKSHOP_PATH}/SalesOrderDetail.csv"
    }
    
    for table_name, path in bronze_tables.items():
        df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(path)
        df.write.format("delta").mode("overwrite").saveAsTable(table_name)
        print(f"✅ {table_name}: {df.count():,} records")
    
    return bronze_tables

# ============================================================
# STEP 2: SILVER LAYER - Data Cleaning & Transformation
# ============================================================

def create_silver_customers():
    """Clean and transform customer data"""
    return (
        spark.table("bronze_customers")
        .withColumn("FirstName", trim(col("FirstName")))
        .withColumn("LastName", trim(col("LastName")))
        .withColumn("EmailAddress", lower(trim(col("EmailAddress"))))
        .dropDuplicates(["CustomerID"])
        .filter(col("CustomerID").isNotNull())
    )

def create_silver_products():
    """Clean and enrich product data with category"""
    products = spark.table("bronze_products")
    categories = spark.table("bronze_product_category")
    
    return (
        products
        .join(categories, "ProductCategoryID", "left")
        .withColumn("Name", trim(col("Name")))
        .withColumn("ProductNumber", upper(trim(col("ProductNumber"))))
        .dropDuplicates(["ProductID"])
        .filter(col("ProductID").isNotNull())
    )

def create_silver_orders():
    """Clean order header and details"""
    orders = (
        spark.table("bronze_orders")
        .filter(col("SalesOrderID").isNotNull())
        .dropDuplicates(["SalesOrderID"])
    )
    
    order_details = (
        spark.table("bronze_order_details")
        .filter(col("SalesOrderDetailID").isNotNull())
        .dropDuplicates(["SalesOrderDetailID"])
    )
    
    return orders, order_details

print("✅ Silver layer functions defined")

In [ ]:
# ============================================================
# STEP 3: GOLD LAYER - Star Schema Dimensions
# ============================================================

def create_dim_date(start_date="2020-01-01", end_date="2025-12-31"):
    """Create date dimension with calendar attributes"""
    from pyspark.sql.functions import explode, sequence, to_date
    
    date_df = spark.sql(f"""
        SELECT explode(sequence(
            to_date('{start_date}'), 
            to_date('{end_date}'), 
            interval 1 day
        )) as full_date
    """)
    
    return (date_df
        .withColumn("date_key", date_format(col("full_date"), "yyyyMMdd").cast("int"))
        .withColumn("year", year(col("full_date")))
        .withColumn("quarter", quarter(col("full_date")))
        .withColumn("month", month(col("full_date")))
        .withColumn("month_name", date_format(col("full_date"), "MMMM"))
        .withColumn("day", dayofmonth(col("full_date")))
        .withColumn("day_of_week", dayofweek(col("full_date")))
        .withColumn("day_name", date_format(col("full_date"), "EEEE"))
        .withColumn("week_of_year", weekofyear(col("full_date")))
        .withColumn("is_weekend", when(dayofweek(col("full_date")).isin(1, 7), True).otherwise(False))
    )

def create_dim_product_scd1(silver_products):
    """Create product dimension with SCD Type 1 (overwrite)"""
    return (silver_products
        .withColumn("product_key", sha2(col("ProductID").cast("string"), 256))
        .select(
            col("product_key"),
            col("ProductID").alias("product_id"),
            col("Name").alias("product_name"),
            col("ProductNumber").alias("product_number"),
            col("Color").alias("color"),
            col("StandardCost").alias("standard_cost"),
            col("ListPrice").alias("list_price"),
            col("Size").alias("size"),
            col("Weight").alias("weight"),
            coalesce(col("ProductCategoryName"), lit("Unknown")).alias("category_name")
        )
    )

def create_dim_customer_scd2(silver_customers):
    """Create customer dimension with SCD Type 2 (history tracking)"""
    return (silver_customers
        .withColumn("customer_key", sha2(col("CustomerID").cast("string"), 256))
        .withColumn("full_name", concat_ws(" ", col("FirstName"), col("LastName")))
        .withColumn("is_current", lit(True))
        .withColumn("valid_from", current_timestamp())
        .withColumn("valid_to", lit(None).cast("timestamp"))
        .withColumn("version", lit(1))
        .select(
            col("customer_key"),
            col("CustomerID").alias("customer_id"),
            col("FirstName").alias("first_name"),
            col("LastName").alias("last_name"),
            col("EmailAddress").alias("email"),
            col("CompanyName").alias("company_name"),
            col("full_name"),
            col("is_current"),
            col("valid_from"),
            col("valid_to"),
            col("version")
        )
    )

print("✅ Gold dimension functions defined")

In [ ]:
# ============================================================
# STEP 4: GOLD LAYER - Fact Table
# ============================================================

def create_fact_sales(silver_orders, silver_order_details):
    """Create fact sales table joining all dimensions"""
    
    # Read dimension tables (current versions only for SCD2)
    dim_customer = spark.table("gold_dim_customer").filter(col("is_current") == True)
    dim_product = spark.table("gold_dim_product")
    dim_date = spark.table("gold_dim_date")
    
    return (silver_order_details
        .join(silver_orders, "SalesOrderID")
        .join(dim_customer.select("customer_key", "customer_id"),
              silver_orders["CustomerID"] == dim_customer["customer_id"], "left")
        .join(dim_product.select("product_key", "product_id"),
              silver_order_details["ProductID"] == dim_product["product_id"], "left")
        .join(dim_date.select("date_key", "full_date"),
              date_format(silver_orders["OrderDate"], "yyyy-MM-dd") == 
              date_format(dim_date["full_date"], "yyyy-MM-dd"), "left")
        .select(
            monotonically_increasing_id().alias("sales_fact_id"),
            col("SalesOrderID").alias("order_id"),
            col("SalesOrderDetailID").alias("order_line_id"),
            col("customer_key"),
            col("product_key"),
            col("date_key"),
            col("OrderQty").alias("quantity"),
            col("UnitPrice").alias("unit_price"),
            col("LineTotal").alias("line_total"),
            col("TotalDue").alias("order_total"),
            col("OrderDate").alias("order_date"),
            col("ShipDate").alias("ship_date")
        )
    )

# ============================================================
# STEP 5: SCD TYPE 2 UPDATE FUNCTION
# ============================================================

def apply_scd2_update(table_name, incoming_df, key_column, compare_columns):
    """
    Apply SCD Type 2 update pattern to a dimension table
    
    Args:
        table_name: Name of the target Delta table
        incoming_df: DataFrame with new/updated records
        key_column: Business key column name (e.g., 'customer_id')
        compare_columns: List of columns to compare for changes
    """
    from delta.tables import DeltaTable
    
    target_table = DeltaTable.forName(spark, table_name)
    
    # Build condition for detecting changes
    change_condition = " OR ".join([
        f"target.{c} != source.{c}" for c in compare_columns
    ])
    
    # Step 1: Expire old records
    target_table.alias("target").merge(
        incoming_df.alias("source"),
        f"target.{key_column} = source.{key_column} AND target.is_current = true"
    ).whenMatchedUpdate(
        condition=change_condition,
        set={
            "is_current": lit(False),
            "valid_to": current_timestamp()
        }
    ).execute()
    
    # Step 2: Insert new versions
    # Get max version for each key
    current_versions = (
        spark.table(table_name)
        .groupBy(key_column)
        .agg(spark_max("version").alias("max_version"))
    )
    
    new_records = (
        incoming_df
        .join(current_versions, key_column, "left")
        .withColumn("version", coalesce(col("max_version") + 1, lit(1)))
        .drop("max_version")
    )
    
    new_records.write.format("delta").mode("append").saveAsTable(table_name)
    
    print(f"✅ SCD Type 2 update applied to {table_name}")

print("✅ Fact table and SCD2 update functions defined")

In [ ]:
# ============================================================
# EXECUTE COMPLETE PYSPARK PIPELINE
# ============================================================

def run_complete_pyspark_pipeline():
    """Execute the complete Star Schema pipeline using PySpark"""
    
    print("=" * 60)
    print("STARTING COMPLETE PYSPARK STAR SCHEMA PIPELINE")
    print("=" * 60)
    
    # Step 1: Bronze Layer
    print("\n📦 STEP 1: Loading Bronze Layer...")
    load_bronze_tables()
    
    # Step 2: Silver Layer
    print("\n🔧 STEP 2: Creating Silver Layer...")
    silver_customers = create_silver_customers()
    silver_customers.write.format("delta").mode("overwrite").saveAsTable("silver_customers")
    print(f"   ✅ silver_customers: {silver_customers.count():,} records")
    
    silver_products = create_silver_products()
    silver_products.write.format("delta").mode("overwrite").saveAsTable("silver_products")
    print(f"   ✅ silver_products: {silver_products.count():,} records")
    
    silver_orders, silver_order_details = create_silver_orders()
    silver_orders.write.format("delta").mode("overwrite").saveAsTable("silver_orders")
    silver_order_details.write.format("delta").mode("overwrite").saveAsTable("silver_order_details")
    print(f"   ✅ silver_orders: {silver_orders.count():,} records")
    print(f"   ✅ silver_order_details: {silver_order_details.count():,} records")
    
    # Step 3: Gold Dimensions
    print("\n⭐ STEP 3: Creating Gold Dimensions...")
    
    dim_date = create_dim_date()
    dim_date.write.format("delta").mode("overwrite").saveAsTable("gold_dim_date")
    print(f"   ✅ gold_dim_date: {dim_date.count():,} records")
    
    dim_product = create_dim_product_scd1(silver_products)
    dim_product.write.format("delta").mode("overwrite").saveAsTable("gold_dim_product")
    print(f"   ✅ gold_dim_product (SCD1): {dim_product.count():,} records")
    
    dim_customer = create_dim_customer_scd2(silver_customers)
    dim_customer.write.format("delta").mode("overwrite").saveAsTable("gold_dim_customer")
    print(f"   ✅ gold_dim_customer (SCD2): {dim_customer.count():,} records")
    
    # Step 4: Gold Fact Table
    print("\n📊 STEP 4: Creating Gold Fact Table...")
    fact_sales = create_fact_sales(silver_orders, silver_order_details)
    fact_sales.write.format("delta").mode("overwrite").saveAsTable("gold_fact_sales")
    print(f"   ✅ gold_fact_sales: {fact_sales.count():,} records")
    
    print("\n" + "=" * 60)
    print("✅ COMPLETE PYSPARK PIPELINE FINISHED SUCCESSFULLY!")
    print("=" * 60)

# Uncomment to run:
# run_complete_pyspark_pipeline()

---
### Solution B: Complete SQL Implementation

The following SQL script implements the complete Star Schema pipeline using pure Spark SQL:

In [ ]:
-- ============================================================
-- SOLUTION B: COMPLETE SQL STAR SCHEMA PIPELINE
-- ============================================================

-- ============================================================
-- STEP 1: BRONZE LAYER - Raw Data Ingestion
-- ============================================================

-- Bronze Customers
CREATE OR REPLACE TABLE bronze_customers_sql
USING CSV
OPTIONS (header = 'true', inferSchema = 'true')
LOCATION '/Volumes/main/default/workshop_data/Customers.csv';

-- Bronze Products
CREATE OR REPLACE TABLE bronze_products_sql
USING CSV
OPTIONS (header = 'true', inferSchema = 'true')
LOCATION '/Volumes/main/default/workshop_data/Product.csv';

-- Bronze Product Category
CREATE OR REPLACE TABLE bronze_product_category_sql
USING CSV
OPTIONS (header = 'true', inferSchema = 'true')
LOCATION '/Volumes/main/default/workshop_data/ProductCategory.csv';

-- Bronze Orders
CREATE OR REPLACE TABLE bronze_orders_sql
USING CSV
OPTIONS (header = 'true', inferSchema = 'true')
LOCATION '/Volumes/main/default/workshop_data/SalesOrderHeader.csv';

-- Bronze Order Details
CREATE OR REPLACE TABLE bronze_order_details_sql
USING CSV
OPTIONS (header = 'true', inferSchema = 'true')
LOCATION '/Volumes/main/default/workshop_data/SalesOrderDetail.csv';

In [ ]:
-- ============================================================
-- STEP 2: SILVER LAYER - Data Cleaning & Transformation
-- ============================================================

-- Silver Customers
CREATE OR REPLACE TABLE silver_customers_sql AS
SELECT DISTINCT
    CustomerID,
    TRIM(FirstName) as FirstName,
    TRIM(LastName) as LastName,
    LOWER(TRIM(EmailAddress)) as EmailAddress,
    Phone,
    CompanyName
FROM bronze_customers_sql
WHERE CustomerID IS NOT NULL;

-- Silver Products (with category enrichment)
CREATE OR REPLACE TABLE silver_products_sql AS
SELECT DISTINCT
    p.ProductID,
    TRIM(p.Name) as Name,
    UPPER(TRIM(p.ProductNumber)) as ProductNumber,
    p.Color,
    p.StandardCost,
    p.ListPrice,
    p.Size,
    p.Weight,
    p.ProductCategoryID,
    COALESCE(c.Name, 'Unknown') as ProductCategoryName
FROM bronze_products_sql p
LEFT JOIN bronze_product_category_sql c ON p.ProductCategoryID = c.ProductCategoryID
WHERE p.ProductID IS NOT NULL;

-- Silver Orders
CREATE OR REPLACE TABLE silver_orders_sql AS
SELECT DISTINCT *
FROM bronze_orders_sql
WHERE SalesOrderID IS NOT NULL;

-- Silver Order Details
CREATE OR REPLACE TABLE silver_order_details_sql AS
SELECT DISTINCT *
FROM bronze_order_details_sql
WHERE SalesOrderDetailID IS NOT NULL;

In [ ]:
-- ============================================================
-- STEP 3: GOLD LAYER - Dimension Tables
-- ============================================================

-- Dim_Date (SCD Type 1 - Full refresh)
CREATE OR REPLACE TABLE gold_dim_date_sql AS
SELECT 
    CAST(date_format(full_date, 'yyyyMMdd') AS INT) as date_key,
    full_date,
    year(full_date) as year,
    quarter(full_date) as quarter,
    month(full_date) as month,
    date_format(full_date, 'MMMM') as month_name,
    dayofmonth(full_date) as day,
    dayofweek(full_date) as day_of_week,
    date_format(full_date, 'EEEE') as day_name,
    weekofyear(full_date) as week_of_year,
    CASE WHEN dayofweek(full_date) IN (1, 7) THEN true ELSE false END as is_weekend
FROM (
    SELECT explode(sequence(
        to_date('2020-01-01'), 
        to_date('2025-12-31'), 
        interval 1 day
    )) as full_date
);

-- Dim_Product (SCD Type 1 - Overwrite)
CREATE OR REPLACE TABLE gold_dim_product_sql AS
SELECT 
    sha2(CAST(ProductID AS STRING), 256) as product_key,
    ProductID as product_id,
    Name as product_name,
    ProductNumber as product_number,
    Color as color,
    StandardCost as standard_cost,
    ListPrice as list_price,
    Size as size,
    Weight as weight,
    ProductCategoryName as category_name
FROM silver_products_sql;

-- Dim_Customer (SCD Type 2 - History tracking)
CREATE OR REPLACE TABLE gold_dim_customer_sql AS
SELECT 
    sha2(CAST(CustomerID AS STRING), 256) as customer_key,
    CustomerID as customer_id,
    FirstName as first_name,
    LastName as last_name,
    EmailAddress as email,
    CompanyName as company_name,
    concat(FirstName, ' ', LastName) as full_name,
    true as is_current,
    current_timestamp() as valid_from,
    CAST(NULL AS TIMESTAMP) as valid_to,
    1 as version
FROM silver_customers_sql;

In [ ]:
-- ============================================================
-- STEP 4: GOLD LAYER - Fact Table
-- ============================================================

CREATE OR REPLACE TABLE gold_fact_sales_sql AS
SELECT 
    monotonically_increasing_id() as sales_fact_id,
    sod.SalesOrderID as order_id,
    sod.SalesOrderDetailID as order_line_id,
    dc.customer_key,
    dp.product_key,
    dd.date_key,
    sod.OrderQty as quantity,
    sod.UnitPrice as unit_price,
    sod.LineTotal as line_total,
    so.TotalDue as order_total,
    so.OrderDate as order_date,
    so.ShipDate as ship_date
FROM silver_order_details_sql sod
JOIN silver_orders_sql so ON sod.SalesOrderID = so.SalesOrderID
LEFT JOIN gold_dim_customer_sql dc 
    ON so.CustomerID = dc.customer_id AND dc.is_current = true
LEFT JOIN gold_dim_product_sql dp 
    ON sod.ProductID = dp.product_id
LEFT JOIN gold_dim_date_sql dd 
    ON date_format(so.OrderDate, 'yyyy-MM-dd') = date_format(dd.full_date, 'yyyy-MM-dd');

In [ ]:
-- ============================================================
-- SCD TYPE 2 UPDATE PATTERN (SQL)
-- ============================================================

-- Create temporary table with incoming updates
CREATE OR REPLACE TEMP VIEW updated_customers_temp AS
SELECT 
    1 as CustomerID, 'John' as FirstName, 'Smith' as LastName, 
    'john.smith@newmail.com' as EmailAddress, 'New Company Inc.' as CompanyName
UNION ALL
SELECT 2, 'Jane', 'Doe', 'jane.doe@email.com', 'Another Corp'
UNION ALL
SELECT 99999, 'New', 'Customer', 'new@email.com', 'Fresh Start LLC';

-- Step 1: Expire old current records where values have changed
MERGE INTO gold_dim_customer_sql AS target
USING updated_customers_temp AS source
ON target.customer_id = source.CustomerID AND target.is_current = true
WHEN MATCHED AND (
    target.email != source.EmailAddress OR 
    target.company_name != source.CompanyName OR
    target.first_name != source.FirstName OR
    target.last_name != source.LastName
) THEN
    UPDATE SET 
        is_current = false,
        valid_to = current_timestamp();

-- Step 2: Insert new versions
INSERT INTO gold_dim_customer_sql
SELECT 
    sha2(CAST(u.CustomerID AS STRING), 256) as customer_key,
    u.CustomerID as customer_id,
    u.FirstName as first_name,
    u.LastName as last_name,
    u.EmailAddress as email,
    u.CompanyName as company_name,
    concat(u.FirstName, ' ', u.LastName) as full_name,
    true as is_current,
    current_timestamp() as valid_from,
    NULL as valid_to,
    COALESCE(
        (SELECT MAX(version) + 1 FROM gold_dim_customer_sql WHERE customer_id = u.CustomerID),
        1
    ) as version
FROM updated_customers_temp u;

---
## 11. Summary & Key Takeaways

### What We Learned

| Topic | Key Concepts |
|-------|--------------|
| **Medallion Architecture** | Bronze (raw) → Silver (cleaned) → Gold (business-ready) |
| **Star Schema** | Central fact table surrounded by dimension tables |
| **SCD Type 1** | Overwrite existing data - no history preserved |
| **SCD Type 2** | Track history with is_current, valid_from, valid_to, version |
| **MERGE Pattern** | Delta Lake's MERGE for upsert and SCD operations |
| **Lakeflow Pipeline** | Standard Spark/Delta approach for building pipelines |

### Best Practices

1. **Use surrogate keys** (hash-based) instead of natural keys for dimensions
2. **Always filter by `is_current = true`** when joining SCD2 dimensions in analytics
3. **Create separate validation queries** to verify data quality at each layer
4. **Partition large fact tables** by date for better performance
5. **Use Delta Lake's MERGE** for efficient incremental updates

### Tables Created

| Layer | Tables |
|-------|--------|
| **Bronze** | bronze_customers, bronze_products, bronze_product_category, bronze_orders, bronze_order_details |
| **Silver** | silver_customers, silver_products, silver_orders, silver_order_details |
| **Gold** | gold_dim_date, gold_dim_product (SCD1), gold_dim_customer (SCD2), gold_fact_sales |

---

### 🎯 Challenge Exercise

Try to extend the Star Schema by:
1. Adding a **Dim_Store** dimension (you can simulate store data)
2. Implementing **SCD Type 3** for one dimension (keeping only current and previous value)
3. Creating additional fact table metrics like **profit margin** or **discount percentage**

In [ ]:
# Import Lakeflow SDP (działa tylko w kontekście potoku Lakeflow)
from pyspark import pipelines as dp
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [ ]:
# Konfiguracja - ścieżki do danych
# W rzeczywistym potoku te wartości pochodzą z konfiguracji pipeline'u
# spark.conf.get("mypipeline.source_path")

# Dla warsztatu użyjemy ścieżek hardcoded
SOURCE_PATH = "/Volumes/training/workshop/data"

---
## 🥉 Krok 1: Bronze Layer - Surowe dane

### Zadanie 1.1: Utwórz tabelę Bronze dla klientów

Bronze layer zawiera dane **dokładnie tak, jak przyszły ze źródła** - bez transformacji.

**Wymagania:**
- Użyj `dp.create_streaming_table()` do utworzenia Streaming Table
- Dodaj `@dp.append_flow` do przepływu danych
- Użyj Auto Loader (`cloudFiles`) do automatycznego wykrywania nowych plików

**Podpowiedź - Lakeflow SDP:**
```python
# Utwórz streaming table
dp.create_streaming_table(
    name="bronze_customers",
    comment="Opis tabeli"
)

# Dodaj append flow
@dp.append_flow(target="bronze_customers")
def bronze_customers_flow():
    return (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .load(path)
    )
```

In [ ]:
# TODO: Utwórz tabelę bronze_customers (Lakeflow SDP)

# Krok 1: Utwórz streaming table
dp.create_streaming_table(
    name="bronze_customers",
    comment="Surowe dane klientów z plików CSV"
)

# Krok 2: Dodaj append flow
@dp.append_flow(target="bronze_customers")
def bronze_customers_flow():
    return (
        spark.readStream
        # TODO: Uzupełnij kod
        # .format("cloudFiles")
        # .option("cloudFiles.format", "csv")
        # .option("header", "true")
        # .load(f"{SOURCE_PATH}/Customers.csv")
    )

### Zadanie 1.2: Utwórz tabelę Bronze dla zamówień

In [ ]:
# TODO: Utwórz tabelę bronze_orders (Lakeflow SDP)

dp.create_streaming_table(
    name="bronze_orders",
    comment="Surowe dane zamówień z plików CSV"
)

@dp.append_flow(target="bronze_orders")
def bronze_orders_flow():
    return (
        spark.readStream
        # TODO: Uzupełnij kod
        # .format("cloudFiles")
        # .option("cloudFiles.format", "csv")
        # .option("header", "true")
        # .load(f"{SOURCE_PATH}/SalesOrderHeader.csv")
    )

### Zadanie 1.3: Utwórz tabelę Bronze dla produktów

In [ ]:
# TODO: Utwórz tabelę bronze_products (Lakeflow SDP)

dp.create_streaming_table(
    name="bronze_products",
    comment="Surowe dane produktów z plików CSV"
)

@dp.append_flow(target="bronze_products")
def bronze_products_flow():
    return (
        spark.readStream
        # TODO: Uzupełnij kod
        # .format("cloudFiles")
        # .option("cloudFiles.format", "csv")
        # .option("header", "true")
        # .load(f"{SOURCE_PATH}/Product.csv")
    )

---
## 🥈 Krok 2: Silver Layer - Data Quality

### Data Quality z Lakeflow Expectations

Lakeflow SDP używa parametru `expect_all_or_drop` w definicji tabeli:

```python
@dp.materialized_view(
    name="silver_customers",
    comment="Oczyszczeni klienci",
    expect_all_or_drop={"valid_id": "CustomerID IS NOT NULL"}
)
def silver_customers():
    return spark.read.table("bronze_customers")
```

**Rodzaje expectations:**
- `expect_all` - loguj naruszenia, ale przepuść dane
- `expect_all_or_drop` - usuń wiersze naruszające regułę
- `expect_all_or_fail` - zatrzymaj potok przy naruszeniu

In [ ]:
# TODO: Utwórz silver_customers z walidacją (Lakeflow SDP)

@dp.materialized_view(
    name="silver_customers",
    comment="Oczyszczone dane klientów z walidacją",
    expect_all_or_drop={"valid_customer_id": "CustomerID IS NOT NULL"}
)
def silver_customers():
    return (
        spark.read.table("bronze_customers")
        .select(
            col("CustomerID").cast("int"),
            col("FirstName"),
            col("LastName"),
            col("EmailAddress"),
            col("Phone"),
            col("CompanyName")
        )
        .withColumn("full_name", concat_ws(" ", col("FirstName"), col("LastName")))
        .dropDuplicates(["CustomerID"])
    )

### Zadanie 2.2: Utwórz tabelę Silver dla zamówień

In [ ]:
# TODO: Utwórz silver_orders z walidacją (Lakeflow SDP)

@dp.materialized_view(
    name="silver_orders",
    comment="Oczyszczone dane zamówień z walidacją",
    expect_all_or_drop={"valid_order": "SalesOrderID IS NOT NULL"}
)
def silver_orders():
    return (
        spark.read.table("bronze_orders")
        .select(
            col("SalesOrderID").cast("int"),
            col("CustomerID").cast("int"),
            col("OrderDate").cast("date"),
            col("ShipDate").cast("date"),
            col("TotalDue").cast("decimal(18,2)"),
            col("Status").cast("int"),
            col("ShipMethod")
        )
        .filter(col("TotalDue") > 0)
    )

### Zadanie 2.3: Utwórz tabelę Silver dla produktów

In [ ]:
# TODO: Utwórz silver_products z walidacją (Lakeflow SDP)

@dp.materialized_view(
    name="silver_products",
    comment="Oczyszczone dane produktów z walidacją",
    expect_all_or_drop={"valid_product": "ProductID IS NOT NULL"}
)
def silver_products():
    return (
        spark.read.table("bronze_products")
        .select(
            col("ProductID").cast("int"),
            col("Name").alias("product_name"),
            col("ProductNumber"),
            col("Color"),
            col("ListPrice").cast("decimal(18,2)"),
            col("StandardCost").cast("decimal(18,2)")
        )
        .filter(col("ListPrice") > 0)
    )

---
## 🥇 Krok 3: Gold Layer - Star Schema z SCD

### SCD Type 2 z Lakeflow AUTO CDC

Lakeflow SDP oferuje wbudowane wsparcie dla SCD przez AUTO CDC:

```python
# Utwórz streaming table dla wymiaru SCD2
dp.create_streaming_table("dim_customer", comment="SCD Type 2")

# Skonfiguruj AUTO CDC
dp.create_auto_cdc_flow(
    target="dim_customer",
    source="silver_customers",
    keys=["customer_id"],
    sequence_by=col("modified_date"),
    stored_as_scd_type="2"
)
```

Zalety AUTO CDC:
- Automatyczne kolumny `__START_AT` i `__END_AT`
- Obsługa zdarzeń out-of-order
- Brak ręcznego kodu MERGE

In [ ]:
# TODO: Utwórz gold_customer_sales (Lakeflow SDP)

@dp.materialized_view(
    name="gold_customer_sales",
    comment="Agregacja sprzedaży per klient"
)
def gold_customer_sales():
    customers = spark.read.table("silver_customers")
    orders = spark.read.table("silver_orders")
    
    return (
        orders
        .join(customers, "CustomerID")
        .groupBy("CustomerID", "full_name")
        .agg(
            sum("TotalDue").alias("total_sales"),
            count("SalesOrderID").alias("order_count"),
            avg("TotalDue").alias("avg_order_value")
        )
    )

---
## 🚀 Krok 4: Deployment

### Instrukcja uruchomienia potoku:

1. **Przejdź do Workflows** → **Delta Live Tables**
2. **Kliknij "Create Pipeline"**
3. **Wypełnij konfigurację:**
   - **Pipeline name:** `workshop_w3_pipeline`
   - **Source code:** Wybierz ten notebook
   - **Target catalog:** Twój katalog
   - **Target schema:** Twój schemat
   - **Cluster mode:** `Enhanced autoscaling`

4. **Dodaj konfigurację** (w sekcji Configuration):
   ```
   mypipeline.source_path = /Volumes/training/workshop/data
   ```

5. **Kliknij "Start"** i obserwuj wizualizację DAG!

In [ ]:
# Ten kod NIE uruchomi DLT - to tylko informacja
print("""
╔══════════════════════════════════════════════════════════════╗
║  📋 INSTRUKCJA URUCHOMIENIA POTOKU DLT                       ║
║                                                              ║
║  1. Przejdź do: Workflows → Delta Live Tables                ║
║  2. Kliknij: Create Pipeline                                 ║
║  3. Wybierz ten notebook jako źródło                         ║
║  4. Skonfiguruj target catalog i schema                      ║
║  5. Kliknij: Start                                           ║
║                                                              ║
║  👀 Obserwuj wizualizację DAG i metryki jakości!            ║
╚══════════════════════════════════════════════════════════════╝
""")

---
---

# 📋 ROZWIĄZANIE

⚠️ **Nie patrz tutaj, dopóki nie spróbujesz sam!** ⚠️

In [ ]:
# ============================================================
# KOMPLETNE ROZWIĄZANIE: Lakeflow SDP Pipeline
# ============================================================

from pyspark import pipelines as dp
from pyspark.sql.functions import *

# ============================================================
# BRONZE LAYER - Streaming Tables
# ============================================================

# Bronze Customers
dp.create_streaming_table("bronze_customers", comment="Raw customer data")

@dp.append_flow(target="bronze_customers")
def bronze_customers_flow():
    return spark.readStream.format("cloudFiles").option("cloudFiles.format", "csv").option("header", "true").load(f"{SOURCE_PATH}/Customers.csv")

# Bronze Orders
dp.create_streaming_table("bronze_orders", comment="Raw order data")

@dp.append_flow(target="bronze_orders")
def bronze_orders_flow():
    return spark.readStream.format("cloudFiles").option("cloudFiles.format", "csv").option("header", "true").load(f"{SOURCE_PATH}/SalesOrderHeader.csv")

# Bronze Products
dp.create_streaming_table("bronze_products", comment="Raw product data")

@dp.append_flow(target="bronze_products")
def bronze_products_flow():
    return spark.readStream.format("cloudFiles").option("cloudFiles.format", "csv").option("header", "true").load(f"{SOURCE_PATH}/Product.csv")

# ============================================================
# SILVER LAYER - Materialized Views
# ============================================================

@dp.materialized_view(
    name="silver_customers",
    comment="Cleaned customer data",
    expect_all_or_drop={"valid_id": "CustomerID IS NOT NULL"}
)
def silver_customers():
    return spark.read.table("bronze_customers").select(
        col("CustomerID").cast("int"),
        concat_ws(" ", col("FirstName"), col("LastName")).alias("full_name"),
        col("EmailAddress"), col("Phone"), col("CompanyName")
    ).dropDuplicates(["CustomerID"])

@dp.materialized_view(name="silver_orders", comment="Cleaned order data")
def silver_orders():
    return spark.read.table("bronze_orders").select(
        col("SalesOrderID").cast("int"), col("CustomerID").cast("int"),
        col("OrderDate").cast("date"), col("TotalDue").cast("decimal(18,2)")
    )

# ============================================================
# GOLD LAYER - SCD z AUTO CDC
# ============================================================

# Dim_Customer SCD Type 2
dp.create_streaming_table("dim_customer", comment="Customer dimension SCD2")
dp.create_auto_cdc_flow(
    target="dim_customer",
    source="silver_customers",
    keys=["CustomerID"],
    sequence_by=col("ModifiedDate"),
    stored_as_scd_type="2"
)

# Fact Sales
@dp.materialized_view(name="fact_sales", comment="Sales fact table")
def fact_sales():
    return (
        spark.read.table("silver_orders")
        .join(spark.read.table("dim_customer").filter(col("__END_AT").isNull()), "CustomerID")
        .select(
            monotonically_increasing_id().alias("sales_key"),
            col("customer_key"), col("SalesOrderID").alias("order_id"),
            col("TotalDue").alias("total_due")
        )
    )

### 🔄 SCD Type 2 - Aktualizacja z MERGE (Symulacja)

Teraz zasymulujemy aktualizację danych klienta i pokażemy jak działa SCD Type 2:

In [ ]:
# ============================================================
# 🔄 SCD TYPE 2 - Aktualizacja z Delta MERGE
# ============================================================

# Symulujemy zmianę danych klienta - klient zmienił firmę
print("📝 Symulacja zmiany danych klienta...")

# Pobierz przykładowego klienta
sample_customer = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}dim_customer") \
    .filter(col("is_current") == True) \
    .limit(1).collect()[0]

print(f"   Przed zmianą: {sample_customer['full_name']} - {sample_customer['company_name']}")

# Utwórz "nowe dane" - symulacja zmiany firmy
from pyspark.sql import Row
updated_data = [(
    sample_customer['customer_id'],
    sample_customer['full_name'],
    sample_customer['first_name'],
    sample_customer['last_name'],
    sample_customer['email'],
    sample_customer['phone'],
    "NOWA FIRMA SP. Z O.O."  # Zmieniona firma!
)]

updated_df = spark.createDataFrame(updated_data, [
    "customer_id", "full_name", "first_name", "last_name", 
    "email", "phone", "company_name"
])

print(f"   Nowa firma: NOWA FIRMA SP. Z O.O.")

In [ ]:
# ============================================================
# SCD TYPE 2 - MERGE w SQL
# ============================================================

# Zarejestruj DataFrame jako temp view
updated_df.createOrReplaceTempView("updated_customers")

# SCD Type 2 wymaga 2 kroków:
# Krok 1: Zamknij stare rekordy (ustaw valid_to i is_current = false)
# Krok 2: Wstaw nowe rekordy

dim_customer_table = f"{catalog}.{schema}.{TABLE_PREFIX}dim_customer"

# KROK 1: Zamknij stare rekordy
spark.sql(f"""
MERGE INTO {dim_customer_table} AS target
USING updated_customers AS source
ON target.customer_id = source.customer_id 
   AND target.is_current = true
   AND target.company_name != source.company_name
WHEN MATCHED THEN UPDATE SET
    target.is_current = false,
    target.valid_to = current_date() - INTERVAL 1 DAY
""")

print("✅ Krok 1: Zamknięto stare rekordy")

# KROK 2: Wstaw nowe rekordy
spark.sql(f"""
INSERT INTO {dim_customer_table}
SELECT 
    (SELECT COALESCE(MAX(customer_key), 0) + 1 FROM {dim_customer_table}) AS customer_key,
    source.customer_id,
    source.full_name,
    source.first_name,
    source.last_name,
    source.email,
    source.phone,
    source.company_name,
    true AS is_current,
    current_date() AS valid_from,
    DATE '9999-12-31' AS valid_to
FROM updated_customers source
WHERE EXISTS (
    SELECT 1 FROM {dim_customer_table} target 
    WHERE target.customer_id = source.customer_id 
    AND target.is_current = false
    AND target.valid_to = current_date() - INTERVAL 1 DAY
)
""")

print("✅ Krok 2: Wstawiono nowe rekordy")

# Weryfikacja
print("\n📊 Historia zmian dla klienta:")
display(spark.sql(f"""
SELECT customer_key, customer_id, full_name, company_name, 
       is_current, valid_from, valid_to
FROM {dim_customer_table}
WHERE customer_id = {sample_customer['customer_id']}
ORDER BY valid_from
"""))

---
### Wymiary Gotowe!

Po utworzeniu wszystkich trzech wymiarów (Dim_Customer, Dim_Product, Dim_Date), możemy przejść do tworzenia tabeli faktów.

In [ ]:
# ============================================================
# ✅ WYMIARY GOLD LAYER GOTOWE!
# ============================================================

print("=" * 60)
print("✅ WSZYSTKIE WYMIARY GOLD LAYER GOTOWE!")
print("=" * 60)
print()
print("📊 Utworzone wymiary:")
print("  • Dim_Customer (SCD Type 2) - z historią zmian")
print("  • Dim_Product (SCD Type 1) - bez historii")
print("  • Dim_Date (SCD Type 1) - wymiar daty")
print()
print("🎯 Następny krok: Fact_Sales")

---
---

## 🥇 Krok 5: Gold Layer - Fact_Sales (Tabela Faktów)

### Tabela Faktów

Tabela faktów to **serce Star Schema** - zawiera:
- **Klucze obce (FK)** do wszystkich wymiarów
- **Miary (Measures)** - wartości liczbowe do analizy
- **Degenerate Dimensions** - np. numer zamówienia

### Struktura Fact_Sales:

| Kolumna | Typ | Opis |
|---------|-----|------|
| `sales_key` | SK | Surrogate Key |
| `customer_key` | FK | Klucz do Dim_Customer |
| `product_key` | FK | Klucz do Dim_Product |
| `date_key` | FK | Klucz do Dim_Date |
| `order_id` | DD | Numer zamówienia |
| `quantity` | Measure | Ilość |
| `unit_price` | Measure | Cena jednostkowa |
| `line_total` | Measure | Wartość linii |

### Zadanie 5.1: Utwórz Fact_Sales

**Wersja PySpark:**

In [ ]:
# ============================================================
# 🥇 GOLD LAYER - Fact_Sales - PySpark
# ============================================================

print("📊 Tworzę: Fact_Sales")

# Wczytaj tabele źródłowe (Silver) i wymiary (Gold)
silver_order_header = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}silver_order_header")
silver_order_detail = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}silver_order_detail")
dim_customer = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}dim_customer").filter(col("is_current") == True)
dim_product = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}dim_product")
dim_date = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}dim_date")

# Złącz Order Header z Order Detail
orders_combined = silver_order_detail.alias("detail") \
    .join(silver_order_header.alias("header"), "SalesOrderID")

# Złącz z wymiarami i utwórz Fact_Sales
fact_sales = orders_combined \
    .join(dim_customer, orders_combined["CustomerID"] == dim_customer["customer_id"], "left") \
    .join(dim_product, orders_combined["ProductID"] == dim_product["product_id"], "left") \
    .withColumn("date_key", date_format(col("OrderDate"), "yyyyMMdd").cast("int")) \
    .withColumn("sales_key", monotonically_increasing_id() + 1) \
    .select(
        col("sales_key"),
        col("customer_key"),
        col("product_key"),
        col("date_key"),
        col("SalesOrderID").alias("order_id"),
        col("SalesOrderDetailID").alias("order_detail_id"),
        col("OrderQty").alias("quantity"),
        col("UnitPrice").alias("unit_price"),
        col("UnitPriceDiscount").alias("discount"),
        col("LineTotal").alias("line_total"),
        col("TotalDue").alias("total_due"),
        col("ShipMethod").alias("ship_method")
    )

# Zapisz jako tabelę Delta
fact_sales.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}fact_sales")

print(f"   ✅ Fact_Sales: {fact_sales.count()} rekordów")
display(fact_sales.limit(5))

**Wersja SQL (równoważna):**

In [ ]:
# ============================================================
# 🥇 GOLD LAYER - Fact_Sales - SQL Version
# ============================================================

spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}fact_sales_sql AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY d.SalesOrderID, d.SalesOrderDetailID) AS sales_key,
    dc.customer_key,
    dp.product_key,
    CAST(DATE_FORMAT(h.OrderDate, 'yyyyMMdd') AS INT) AS date_key,
    d.SalesOrderID AS order_id,
    d.SalesOrderDetailID AS order_detail_id,
    d.OrderQty AS quantity,
    d.UnitPrice AS unit_price,
    d.UnitPriceDiscount AS discount,
    d.LineTotal AS line_total,
    h.TotalDue AS total_due,
    h.ShipMethod AS ship_method
FROM {catalog}.{schema}.{TABLE_PREFIX}silver_order_detail d
JOIN {catalog}.{schema}.{TABLE_PREFIX}silver_order_header h 
    ON d.SalesOrderID = h.SalesOrderID
LEFT JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_customer dc 
    ON h.CustomerID = dc.customer_id AND dc.is_current = true
LEFT JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_product dp 
    ON d.ProductID = dp.product_id
""")

print("✅ Fact_Sales (SQL) utworzony!")

---
---

## 🎯 Krok 6: Weryfikacja Star Schema

Sprawdźmy, czy nasz model Star Schema działa poprawnie:

In [ ]:
# ============================================================
# 🎯 WERYFIKACJA STAR SCHEMA
# ============================================================

print("📊 PODSUMOWANIE STAR SCHEMA:")
print("=" * 60)

# Podsumowanie tabel - 3 wymiary + 1 fakt
tables = [
    ("Dim_Customer", f"{TABLE_PREFIX}dim_customer"),
    ("Dim_Product", f"{TABLE_PREFIX}dim_product"),
    ("Dim_Date", f"{TABLE_PREFIX}dim_date"),
    ("Fact_Sales", f"{TABLE_PREFIX}fact_sales")
]

for name, table in tables:
    count = spark.table(f"{catalog}.{schema}.{table}").count()
    print(f"  {name}: {count:,} rekordów")

print("=" * 60)

In [ ]:
# ============================================================
# 📈 PRZYKŁADOWE ZAPYTANIA ANALITYCZNE
# ============================================================

print("📈 PRZYKŁADOWE ZAPYTANIA NA STAR SCHEMA:")
print()

# Zapytanie 1: Sprzedaż per klient
print("1️⃣ TOP 10 klientów wg wartości sprzedaży:")
display(spark.sql(f"""
SELECT 
    c.full_name,
    c.company_name,
    SUM(f.line_total) AS total_sales,
    COUNT(DISTINCT f.order_id) AS order_count
FROM {catalog}.{schema}.{TABLE_PREFIX}fact_sales f
JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_customer c 
    ON f.customer_key = c.customer_key
GROUP BY c.full_name, c.company_name
ORDER BY total_sales DESC
LIMIT 10
"""))

In [ ]:
# Zapytanie 2: Sprzedaż per kategoria produktu
print("2️⃣ Sprzedaż per kategoria produktu:")
display(spark.sql(f"""
SELECT 
    p.category_name,
    SUM(f.line_total) AS total_sales,
    SUM(f.quantity) AS total_quantity,
    AVG(f.unit_price) AS avg_price
FROM {catalog}.{schema}.{TABLE_PREFIX}fact_sales f
JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_product p 
    ON f.product_key = p.product_key
WHERE p.category_name IS NOT NULL
GROUP BY p.category_name
ORDER BY total_sales DESC
"""))

In [ ]:
# Zapytanie 3: Sprzedaż per miesiąc (z Dim_Date)
print("3️⃣ Trend sprzedaży miesięcznej:")
display(spark.sql(f"""
SELECT 
    d.year,
    d.month,
    d.month_name,
    SUM(f.line_total) AS total_sales,
    COUNT(DISTINCT f.order_id) AS order_count
FROM {catalog}.{schema}.{TABLE_PREFIX}fact_sales f
JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_date d 
    ON f.date_key = d.date_key
GROUP BY d.year, d.month, d.month_name
ORDER BY d.year, d.month
"""))

In [ ]:
# Zapytanie 4: Analiza wielowymiarowa (CUBE)
print("4️⃣ Analiza wielowymiarowa - kategorie i lata:")
display(spark.sql(f"""
SELECT 
    p.category_name,
    d.year,
    SUM(f.line_total) AS total_sales,
    COUNT(DISTINCT f.order_id) AS order_count,
    AVG(f.line_total) AS avg_order_value
FROM {catalog}.{schema}.{TABLE_PREFIX}fact_sales f
JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_product p 
    ON f.product_key = p.product_key
JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_date d 
    ON f.date_key = d.date_key
WHERE p.category_name IS NOT NULL
GROUP BY p.category_name, d.year
ORDER BY p.category_name, d.year
"""))

---
---

# 📋 ROZWIĄZANIE KOMPLETNE

⚠️ **Nie patrz tutaj, dopóki nie spróbujesz sam!** ⚠️

Poniżej znajdziesz kompletny kod do budowy Star Schema w dwóch wersjach:
1. **PySpark** - programistyczne podejście
2. **SQL** - deklaratywne podejście

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║                                                                              ║
# ║   📋 FULL SOLUTION - PYSPARK VERSION                                         ║
# ║   Workshop 3: Lakeflow Pipeline - Star Schema with SCD                       ║
# ║                                                                              ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta

# --- CONFIGURATION ---
catalog = spark.conf.get("spark.databricks.unityCatalog.catalog", "training")
schema = spark.conf.get("spark.databricks.unityCatalog.schema", "workshop")
volume_path = f"/Volumes/{catalog}/{schema}/data"
TABLE_PREFIX = "w3_"

print("=" * 70)
print("🏭 FULL SOLUTION - STAR SCHEMA WITH SCD (PySpark)")
print("=" * 70)

# ============================================================
# 🥉 BRONZE LAYER
# ============================================================
print("\n🥉 BRONZE LAYER")
print("-" * 50)

source_files = ["Customers.csv", "Product.csv", 
                "ProductCategory.csv", "SalesOrderHeader.csv", "SalesOrderDetail.csv"]

for file_name in source_files:
    table_name = f"{TABLE_PREFIX}bronze_{file_name.replace('.csv', '').lower()}"
    df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"{volume_path}/{file_name}")
    df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{table_name}")
    print(f"  ✅ {table_name}: {df.count()} records")

# ============================================================
# 🥈 SILVER LAYER
# ============================================================
print("\n🥈 SILVER LAYER")
print("-" * 50)

# Silver Customers
silver_customers = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_customers") \
    .select(col("CustomerID").cast("int"), "FirstName", "LastName", "EmailAddress", "Phone", "CompanyName", col("ModifiedDate").cast("timestamp")) \
    .withColumn("full_name", concat_ws(" ", col("FirstName"), col("LastName"))) \
    .filter(col("EmailAddress").isNotNull()).dropDuplicates(["CustomerID"])
silver_customers.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}silver_customers")
print(f"  ✅ silver_customers: {silver_customers.count()} records")

# Silver Products (with category)
bronze_products = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_product")
bronze_categories = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_productcategory")
silver_products = bronze_products.join(bronze_categories, bronze_products.ProductCategoryID == bronze_categories.ProductCategoryID, "left") \
    .select(bronze_products.ProductID.cast("int"), bronze_products.Name.alias("product_name"), bronze_products.ProductNumber, 
            bronze_products.Color, bronze_products.ListPrice.cast("decimal(18,2)"), bronze_products.StandardCost.cast("decimal(18,2)"),
            bronze_categories.Name.alias("category_name"), bronze_products.ModifiedDate.cast("timestamp")) \
    .filter(col("ProductID").isNotNull()).dropDuplicates(["ProductID"])
silver_products.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}silver_products")
print(f"  ✅ silver_products: {silver_products.count()} records")

# Silver Orders
bronze_orders = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_salesorderheader")
bronze_details = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_salesorderdetail")

silver_order_header = bronze_orders.select(col("SalesOrderID").cast("int"), col("CustomerID").cast("int"),
    col("OrderDate").cast("date"), col("ShipDate").cast("date"), col("TotalDue").cast("decimal(18,2)"), col("Status").cast("int"), "ShipMethod") \
    .filter(col("SalesOrderID").isNotNull())
silver_order_header.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}silver_order_header")
print(f"  ✅ silver_order_header: {silver_order_header.count()} records")

silver_order_detail = bronze_details.select(col("SalesOrderID").cast("int"), col("SalesOrderDetailID").cast("int"), col("ProductID").cast("int"),
    col("OrderQty").cast("int"), col("UnitPrice").cast("decimal(18,2)"), col("UnitPriceDiscount").cast("decimal(5,2)"), col("LineTotal").cast("decimal(18,2)")) \
    .filter(col("SalesOrderDetailID").isNotNull())
silver_order_detail.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}silver_order_detail")
print(f"  ✅ silver_order_detail: {silver_order_detail.count()} records")

# ============================================================
# 🥇 GOLD LAYER - DIMENSIONS
# ============================================================
print("\n🥇 GOLD LAYER - DIMENSIONS")
print("-" * 50)

# Dim_Product (SCD Type 1)
dim_product = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}silver_products") \
    .withColumn("product_key", monotonically_increasing_id() + 1) \
    .select(col("product_key"), col("ProductID").alias("product_id"), col("product_name"), col("ProductNumber").alias("product_number"),
            col("Color").alias("color"), col("category_name"), col("ListPrice").alias("list_price"), col("StandardCost").alias("standard_cost"),
            current_timestamp().alias("load_timestamp"))
dim_product.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}dim_product")
print(f"  ✅ dim_product (SCD1): {dim_product.count()} records")

# Dim_Date (SCD Type 1)
start_date, end_date = datetime(2022, 1, 1), datetime(2026, 12, 31)
date_range = [(start_date + timedelta(days=x)).strftime('%Y-%m-%d') for x in range((end_date - start_date).days + 1)]
dim_date = spark.createDataFrame([(d,) for d in date_range], ["full_date"]) \
    .withColumn("full_date", col("full_date").cast("date")) \
    .withColumn("date_key", date_format(col("full_date"), "yyyyMMdd").cast("int")) \
    .withColumn("year", year(col("full_date"))).withColumn("month", month(col("full_date"))).withColumn("day", dayofmonth(col("full_date"))) \
    .withColumn("quarter", quarter(col("full_date"))).withColumn("day_of_week", dayofweek(col("full_date"))) \
    .withColumn("day_name", date_format(col("full_date"), "EEEE")).withColumn("month_name", date_format(col("full_date"), "MMMM")) \
    .withColumn("is_weekend", when(dayofweek(col("full_date")).isin([1, 7]), True).otherwise(False)) \
    .select("date_key", "full_date", "year", "month", "month_name", "day", "quarter", "day_of_week", "day_name", "is_weekend")
dim_date.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}dim_date")
print(f"  ✅ dim_date (SCD1): {dim_date.count()} records")

# Dim_Customer (SCD Type 2)
dim_customer = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}silver_customers") \
    .withColumn("customer_key", monotonically_increasing_id() + 1) \
    .withColumn("is_current", lit(True)).withColumn("valid_from", current_date()).withColumn("valid_to", lit("9999-12-31").cast("date")) \
    .select(col("customer_key"), col("CustomerID").alias("customer_id"), col("full_name"), col("FirstName").alias("first_name"),
            col("LastName").alias("last_name"), col("EmailAddress").alias("email"), col("Phone").alias("phone"),
            col("CompanyName").alias("company_name"), col("is_current"), col("valid_from"), col("valid_to"))
dim_customer.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}dim_customer")
print(f"  ✅ dim_customer (SCD2): {dim_customer.count()} records")

# ============================================================
# 🥇 GOLD LAYER - FACT_SALES
# ============================================================
print("\n🥇 GOLD LAYER - FACT TABLE")
print("-" * 50)

silver_order_header = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}silver_order_header")
silver_order_detail = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}silver_order_detail")
dim_customer = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}dim_customer").filter(col("is_current") == True)
dim_product = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}dim_product")

orders_combined = silver_order_detail.alias("d").join(silver_order_header.alias("h"), "SalesOrderID")

fact_sales = orders_combined \
    .join(dim_customer, orders_combined["CustomerID"] == dim_customer["customer_id"], "left") \
    .join(dim_product, orders_combined["ProductID"] == dim_product["product_id"], "left") \
    .withColumn("date_key", date_format(col("OrderDate"), "yyyyMMdd").cast("int")) \
    .withColumn("sales_key", monotonically_increasing_id() + 1) \
    .select(col("sales_key"), col("customer_key"), col("product_key"), col("date_key"),
            col("SalesOrderID").alias("order_id"), col("SalesOrderDetailID").alias("order_detail_id"),
            col("OrderQty").alias("quantity"), col("UnitPrice").alias("unit_price"), col("UnitPriceDiscount").alias("discount"),
            col("LineTotal").alias("line_total"), col("TotalDue").alias("total_due"), col("ShipMethod").alias("ship_method"))

fact_sales.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{TABLE_PREFIX}fact_sales")
print(f"  ✅ fact_sales: {fact_sales.count()} records")

print("\n" + "=" * 70)
print("🎉 STAR SCHEMA COMPLETE!")
print("=" * 70)

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║                                                                              ║
# ║   📋 FULL SOLUTION - SQL VERSION                                             ║
# ║   Workshop 3: Lakeflow Pipeline - Star Schema with SCD                       ║
# ║                                                                              ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# --- CONFIGURATION ---
catalog = spark.conf.get("spark.databricks.unityCatalog.catalog", "training")
schema = spark.conf.get("spark.databricks.unityCatalog.schema", "workshop")
volume_path = f"/Volumes/{catalog}/{schema}/data"
TABLE_PREFIX = "w3_sql_"

print("=" * 70)
print("🏭 FULL SOLUTION - STAR SCHEMA WITH SCD (SQL)")
print("=" * 70)

# ============================================================
# 🥉 BRONZE LAYER (SQL)
# ============================================================
print("\n🥉 BRONZE LAYER")
print("-" * 50)

source_files = {
    "Customers": "Customers.csv",
    "Product": "Product.csv",
    "ProductCategory": "ProductCategory.csv",
    "SalesOrderHeader": "SalesOrderHeader.csv",
    "SalesOrderDetail": "SalesOrderDetail.csv"
}

for table_name, file_name in source_files.items():
    spark.sql(f"""
        CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}bronze_{table_name.lower()}
        USING CSV
        OPTIONS (
            path '{volume_path}/{file_name}',
            header 'true',
            inferSchema 'true'
        )
    """)
    count = spark.table(f"{catalog}.{schema}.{TABLE_PREFIX}bronze_{table_name.lower()}").count()
    print(f"  ✅ bronze_{table_name.lower()}: {count} records")

# ============================================================
# 🥈 SILVER LAYER (SQL)
# ============================================================
print("\n🥈 SILVER LAYER")
print("-" * 50)

# Silver Customers
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}silver_customers AS
SELECT 
    CAST(CustomerID AS INT) AS CustomerID,
    FirstName,
    LastName,
    CONCAT_WS(' ', FirstName, LastName) AS full_name,
    EmailAddress,
    Phone,
    CompanyName,
    CAST(ModifiedDate AS TIMESTAMP) AS ModifiedDate
FROM {catalog}.{schema}.{TABLE_PREFIX}bronze_customers
WHERE EmailAddress IS NOT NULL
""")
print(f"  ✅ silver_customers: {spark.table(f'{catalog}.{schema}.{TABLE_PREFIX}silver_customers').count()} records")

# Silver Products (with category)
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}silver_products AS
SELECT 
    CAST(p.ProductID AS INT) AS ProductID,
    p.Name AS product_name,
    p.ProductNumber,
    p.Color,
    CAST(p.ListPrice AS DECIMAL(18,2)) AS ListPrice,
    CAST(p.StandardCost AS DECIMAL(18,2)) AS StandardCost,
    c.Name AS category_name,
    CAST(p.ModifiedDate AS TIMESTAMP) AS ModifiedDate
FROM {catalog}.{schema}.{TABLE_PREFIX}bronze_product p
LEFT JOIN {catalog}.{schema}.{TABLE_PREFIX}bronze_productcategory c 
    ON p.ProductCategoryID = c.ProductCategoryID
WHERE p.ProductID IS NOT NULL
""")
print(f"  ✅ silver_products: {spark.table(f'{catalog}.{schema}.{TABLE_PREFIX}silver_products').count()} records")

# Silver Order Header
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}silver_order_header AS
SELECT 
    CAST(SalesOrderID AS INT) AS SalesOrderID,
    CAST(CustomerID AS INT) AS CustomerID,
    CAST(OrderDate AS DATE) AS OrderDate,
    CAST(ShipDate AS DATE) AS ShipDate,
    CAST(TotalDue AS DECIMAL(18,2)) AS TotalDue,
    CAST(Status AS INT) AS Status,
    ShipMethod
FROM {catalog}.{schema}.{TABLE_PREFIX}bronze_salesorderheader
WHERE SalesOrderID IS NOT NULL
""")
print(f"  ✅ silver_order_header: {spark.table(f'{catalog}.{schema}.{TABLE_PREFIX}silver_order_header').count()} records")

# Silver Order Detail
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}silver_order_detail AS
SELECT 
    CAST(SalesOrderID AS INT) AS SalesOrderID,
    CAST(SalesOrderDetailID AS INT) AS SalesOrderDetailID,
    CAST(ProductID AS INT) AS ProductID,
    CAST(OrderQty AS INT) AS OrderQty,
    CAST(UnitPrice AS DECIMAL(18,2)) AS UnitPrice,
    CAST(UnitPriceDiscount AS DECIMAL(5,2)) AS UnitPriceDiscount,
    CAST(LineTotal AS DECIMAL(18,2)) AS LineTotal
FROM {catalog}.{schema}.{TABLE_PREFIX}bronze_salesorderdetail
WHERE SalesOrderDetailID IS NOT NULL
""")
print(f"  ✅ silver_order_detail: {spark.table(f'{catalog}.{schema}.{TABLE_PREFIX}silver_order_detail').count()} records")

# ============================================================
# 🥇 GOLD LAYER - DIMENSIONS (SQL)
# ============================================================
print("\n🥇 GOLD LAYER - DIMENSIONS")
print("-" * 50)

# Dim_Product (SCD Type 1)
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}dim_product AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY ProductID) AS product_key,
    ProductID AS product_id,
    product_name,
    ProductNumber AS product_number,
    Color AS color,
    category_name,
    ListPrice AS list_price,
    StandardCost AS standard_cost,
    current_timestamp() AS load_timestamp
FROM {catalog}.{schema}.{TABLE_PREFIX}silver_products
""")
print(f"  ✅ dim_product (SCD1): {spark.table(f'{catalog}.{schema}.{TABLE_PREFIX}dim_product').count()} records")

# Dim_Date (SCD Type 1)
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}dim_date AS
SELECT 
    CAST(DATE_FORMAT(date_col, 'yyyyMMdd') AS INT) AS date_key,
    date_col AS full_date,
    YEAR(date_col) AS year,
    MONTH(date_col) AS month,
    DATE_FORMAT(date_col, 'MMMM') AS month_name,
    DAY(date_col) AS day,
    QUARTER(date_col) AS quarter,
    DAYOFWEEK(date_col) AS day_of_week,
    DATE_FORMAT(date_col, 'EEEE') AS day_name,
    CASE WHEN DAYOFWEEK(date_col) IN (1, 7) THEN true ELSE false END AS is_weekend
FROM (
    SELECT EXPLODE(SEQUENCE(DATE '2022-01-01', DATE '2026-12-31', INTERVAL 1 DAY)) AS date_col
)
""")
print(f"  ✅ dim_date (SCD1): {spark.table(f'{catalog}.{schema}.{TABLE_PREFIX}dim_date').count()} records")

# Dim_Customer (SCD Type 2)
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}dim_customer AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY CustomerID) AS customer_key,
    CustomerID AS customer_id,
    full_name,
    FirstName AS first_name,
    LastName AS last_name,
    EmailAddress AS email,
    Phone AS phone,
    CompanyName AS company_name,
    true AS is_current,
    current_date() AS valid_from,
    DATE '9999-12-31' AS valid_to
FROM {catalog}.{schema}.{TABLE_PREFIX}silver_customers
""")
print(f"  ✅ dim_customer (SCD2): {spark.table(f'{catalog}.{schema}.{TABLE_PREFIX}dim_customer').count()} records")

# ============================================================
# 🥇 GOLD LAYER - FACT_SALES (SQL)
# ============================================================
print("\n🥇 GOLD LAYER - FACT TABLE")
print("-" * 50)

spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{schema}.{TABLE_PREFIX}fact_sales AS
SELECT 
    ROW_NUMBER() OVER (ORDER BY d.SalesOrderID, d.SalesOrderDetailID) AS sales_key,
    dc.customer_key,
    dp.product_key,
    CAST(DATE_FORMAT(h.OrderDate, 'yyyyMMdd') AS INT) AS date_key,
    d.SalesOrderID AS order_id,
    d.SalesOrderDetailID AS order_detail_id,
    d.OrderQty AS quantity,
    d.UnitPrice AS unit_price,
    d.UnitPriceDiscount AS discount,
    d.LineTotal AS line_total,
    h.TotalDue AS total_due,
    h.ShipMethod AS ship_method
FROM {catalog}.{schema}.{TABLE_PREFIX}silver_order_detail d
JOIN {catalog}.{schema}.{TABLE_PREFIX}silver_order_header h 
    ON d.SalesOrderID = h.SalesOrderID
LEFT JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_customer dc 
    ON h.CustomerID = dc.customer_id AND dc.is_current = true
LEFT JOIN {catalog}.{schema}.{TABLE_PREFIX}dim_product dp 
    ON d.ProductID = dp.product_id
""")
print(f"  ✅ fact_sales: {spark.table(f'{catalog}.{schema}.{TABLE_PREFIX}fact_sales').count()} records")

print("\n" + "=" * 70)
print("🎉 STAR SCHEMA (SQL VERSION) COMPLETE!")
print("=" * 70)

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║                                                                              ║
# ║   📋 BONUS: SCD TYPE 2 MERGE - AKTUALIZACJA WYMIARU                          ║
# ║                                                                              ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║  SCD TYPE 2 - PATTERN AKTUALIZACJI Z DELTA MERGE                             ║
╚══════════════════════════════════════════════════════════════════════════════╝

Gdy przychodzą nowe dane klientów (np. klient zmienił firmę), wykonujemy 
dwuetapowy proces MERGE:

KROK 1: Zamknij stare rekordy (ustaw is_current=false, valid_to=data-1)
========================================================================

MERGE INTO dim_customer AS target
USING (
    SELECT * FROM updated_customers
    WHERE EXISTS (
        SELECT 1 FROM dim_customer d 
        WHERE d.customer_id = updated_customers.customer_id 
        AND d.is_current = true
        AND (d.company_name != updated_customers.company_name 
             OR d.email != updated_customers.email)
    )
) AS source
ON target.customer_id = source.customer_id 
   AND target.is_current = true
WHEN MATCHED THEN UPDATE SET
    target.is_current = false,
    target.valid_to = current_date() - INTERVAL 1 DAY


KROK 2: Wstaw nowe wersje rekordów
==================================

INSERT INTO dim_customer
SELECT 
    (SELECT MAX(customer_key) + ROW_NUMBER() OVER (ORDER BY customer_id) 
     FROM dim_customer) AS customer_key,
    customer_id,
    full_name,
    first_name,
    last_name,
    email,
    phone,
    company_name,
    true AS is_current,
    current_date() AS valid_from,
    DATE '9999-12-31' AS valid_to
FROM updated_customers source
WHERE EXISTS (
    SELECT 1 FROM dim_customer target 
    WHERE target.customer_id = source.customer_id 
    AND target.is_current = false
    AND target.valid_to = current_date() - INTERVAL 1 DAY
)


ALTERNATYWNIE - JEDEN MERGE Z WIELOMA AKCJAMI (Databricks):
===========================================================

MERGE INTO dim_customer AS target
USING updated_customers AS source
ON target.customer_id = source.customer_id AND target.is_current = true

-- Zamknij stary rekord gdy są zmiany
WHEN MATCHED AND (target.company_name != source.company_name) THEN
    UPDATE SET 
        is_current = false,
        valid_to = current_date() - INTERVAL 1 DAY

-- Wstaw nowy rekord (jako second pass)
WHEN NOT MATCHED THEN
    INSERT (customer_key, customer_id, full_name, first_name, last_name, 
            email, phone, company_name, is_current, valid_from, valid_to)
    VALUES (
        (SELECT COALESCE(MAX(customer_key), 0) + 1 FROM dim_customer),
        source.customer_id, source.full_name, source.first_name, source.last_name,
        source.email, source.phone, source.company_name,
        true, current_date(), DATE '9999-12-31'
    )
""")

print("\n✅ Workshop 3 zakończony!")
print("=" * 70)
print("📊 Stworzyłeś kompletny model Star Schema z:")
print("   • 3 wymiarami (Dim_Customer, Dim_Product, Dim_Date)")
print("   • 1 tabelą faktów (Fact_Sales)")
print("   • SCD Type 1 (Dim_Product, Dim_Date)")
print("   • SCD Type 2 (Dim_Customer)")
print("   • Implementacją w PySpark i SQL")
print("=" * 70)